In [2]:
from itertools import combinations
from functools import partial
from pprint import pprint as pp

def generate_input(attrs_by_obj):
    objs_by_attr = dict()
    for obj, attrs in attrs_by_obj.iteritems():
        for attr in attrs:
            if attr not in objs_by_attr:
                objs_by_attr[attr] = set()
            objs_by_attr[attr].add(obj)
    
    attributes = reduce(lambda a, b: a.union(b), attrs_by_obj.values())
    objects = sorted(attrs_by_obj.keys())
    
    attributes_set = set(attributes)
    objects_set = set(objects)
    
    return attributes, attributes_set, objs_by_attr, objects, objects_set, attrs_by_obj
    

def obj_closure(objects, all_attributes, attrs_by_obj):
    m = set(all_attributes)
    for obj in objects:
        m = m.intersection(attrs_by_obj[obj])
    return m

def attr_closure(attributes, all_objects, objs_by_attr):
    g = set(all_objects)
    for attr in attributes:
        g = g.intersection(objs_by_attr[attr])
    return g


def closure(attributes, all_attributes, all_objects, attrs_by_obj, objs_by_attr):
    return obj_closure(attr_closure(attributes, all_objects, objs_by_attr), all_attributes, attrs_by_obj)


def get_pseudo_intents(attributes_set, get_closure):
    pseudo_intents = list()
    
    for i in xrange(0, len(attributes_set) + 1):
        for subset in list(combinations(attributes_set, i)):
            subset = set(subset)            
            if subset != get_closure(subset):
                is_pseudo_intent = True
                for pseudo_intent in pseudo_intents:
                    if pseudo_intent < subset:
                        if not get_closure(pseudo_intent) <= subset:
                            is_pseudo_intent = False
                            break
                if is_pseudo_intent:
                    pseudo_intents.append(subset)
    return pseudo_intents

def get_preclosure(intent_set, pseudo_intents, get_closure):
    prev_intents = None
    cur_intents = intent_set
    while prev_intents != cur_intents:
        pseudo_intents_union = set()
        for pseudo_intent in pseudo_intents:
            if pseudo_intent < cur_intents:
                pseudo_intents_union = pseudo_intents_union.union(get_closure(pseudo_intent))
        prev_intents = cur_intents
        cur_intents = cur_intents.union(pseudo_intents_union)
    return cur_intents

def next_closure(attribute_subset, attributes, get_closure):
    s = set(attribute_subset)
    for a in reversed(attributes):
        if a in s:
            s = s.difference(set([a]))
        else:
            b = get_closure(s.union(set([a])))
            diff = b.difference(s)
            contains_smaller = any(attributes.index(i) < attributes.index(a) for i in diff)
            if not contains_smaller:
                return b
    return set(attributes)

def get_canonical_basis(attributes, get_closure, get_preclosure):
    el = []
    a = set()
    attributes_set = set(attributes)
    while a != attributes_set:
        a_closure = get_closure(a) 
        if a != a_closure:
            el.append([a, '->', a_closure.difference(a)])
        a = next_closure(a, attributes, get_preclosure)
    return el

    
def calculate_answer(attrs_by_obj):
    all_attributes, attributes_set, objs_by_attr, all_objects, objects_set, attrs_by_obj = generate_input(attrs_by_obj)
    
    prefilled_closure = lambda attributes: closure(attributes, all_attributes, all_objects, attrs_by_obj, objs_by_attr) 
    pseudo_intents = get_pseudo_intents(attributes_set, prefilled_closure)
    preclosure = lambda intent_set: get_preclosure(intent_set, pseudo_intents, prefilled_closure)
    basis = get_canonical_basis(sorted(all_attributes), prefilled_closure, preclosure)
    pp(len(basis))
    pp(basis)

6
[[set(['6']), '->', set(['2', '5'])],
 [set(['5']), '->', set(['2', '6'])],
 [set(['4']), '->', set(['2'])],
 [set(['3']), '->', set(['2'])],
 [set(['2', '7']), '->', set(['1', '3', '4', '5', '6'])],
 [set(['1']), '->', set(['2'])]]


In [16]:
def question3():
    attrs_by_obj = {
        '1': set(['1', '2', '3', '4']),
        '2': set(['1', '2', '3', '4', '5', '6']),
        '3': set(['1', '2', '3', '4', '5', '6']),
        '4': set(['1', '2', '3', '4', '5', '6']),
        '5': set(['2', '3', '4', '5', '6']),
        '6': set(['2', '3', '4', '5', '6']),
        
        '7': set([]),
        '8': set(['1', '2', '4', '5', '6']),
        '9': set(['1', '2', '3', '5', '6']),
    }
    calculate_answer(attrs_by_obj)

    
question3()
# Answer is 3 because we need all 3 of them to get the same canonical basis

5
[[set(['6']), '->', set(['2', '5'])],
 [set(['5']), '->', set(['2', '6'])],
 [set(['4']), '->', set(['2'])],
 [set(['3']), '->', set(['2'])],
 [set(['1']), '->', set(['2'])]]


In [3]:
def question4():
    attrs_by_obj = {
        '1': set(['r', 'wc', 'a']),
        '2': set(['d', 'a']),
        '3': set(['r', 'wc', 'a', 'tr']),
        '4': set(['r', 'wc', 'a', 'tr', 'to']),
        '5': set(['sc', 'r', 'wc', 'to']),
        '6': set(['d', 'a', 'tr']),
        '7': set(['d']),
        '8': set(['wc', 'a']),
        '9': set(['sc', 'r', 'wc']),
        '10': set(['sc', 'r', 'wc', 'tr']),
        '11': set(['d', 'tr']),
        '12': set(['r', 'wc', 'a', 'to']),
    }
    calculate_answer(attrs_by_obj)

question4()
# Answer is 7

7
[[set(['tr', 'wc']), '->', set(['r'])],
 [set(['to']), '->', set(['r', 'wc'])],
 [set(['sc']), '->', set(['r', 'wc'])],
 [set(['r']), '->', set(['wc'])],
 [set(['r', 'to', 'tr', 'wc']), '->', set(['a'])],
 [set(['d', 'wc']), '->', set(['a', 'r', 'sc', 'to', 'tr'])],
 [set(['a', 'r', 'sc', 'wc']), '->', set(['d', 'to', 'tr'])]]
